In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import json
import os,sys
os.environ['BASE_DIR'] = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
os.environ['COMMON_DIR'] = os.path.join(os.environ['BASE_DIR'], "geco_commons")
sys.path.append(os.path.join(os.environ['COMMON_DIR'],"util" ))
from daylib import daylib  as dl

In [2]:


import master
from master import *
from datetime import datetime as dt

/usr/local/lib/python3.7/dist-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):
/usr/local/lib/python3.7/dist-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):


Log path = /root/src/aleister/log/logging.log


In [3]:
# args=['--execute_mode',"prepro",'--config_source','ini','--model_id','test001',
#       '-train_sd','20210101','-train_ed','20210103',
#       '-valid_sd','20210104','-train_ed','20210105',
#       '-test_sd','20210106','-test_ed','20210107']
args=['--symbol','BTC', '--execute_mode',"gtrain",
      '--model_id','test002','--model_name','slstm',
      '--config_source','ini',
      '--user',"Misaka00001", '--source','ipynb',
      "--general_config_mode", "DOCKER","--private_api_mode","DEFAULT",
      '-train_sd','20210101','-train_ed','20210103',
      '-valid_sd','20210104','-valid_ed','20210105',
      '-test_sd','20210106','-test_ed','20210107']

In [4]:
parser = make_parser()
arg_dict = vars(parser.parse_args(args))

# meta
_id = arg_dict["model_id"]
general_config_mode = arg_dict["general_config_mode"].upper()
private_api_mode = arg_dict["private_api_mode"].upper()
sym = arg_dict["symbol"]

model_name = arg_dict["model_name"].upper()

om = OperateMaster()
om.load_meta(_id, model_name, sym, general_config_mode, private_api_mode)
om.init_prepro()

# mlflow
mlflow_tags = {
    MLFLOW_USER: arg_dict["user"],
    MLFLOW_SOURCE_NAME: arg_dict["source"],
    MLFLOW_RUN_NAME: f"TRAIN_{dt.now().strftime('%y%m%d%H%M%s')}"
}
mlflow_client_kwargs = {
    "tracking_uri": os.environ['MLFLOW_TRACKING_URI']
}
om.set_mlflow_settings(mlflow_client_kwargs, mlflow_tags)

# load conofigs into each module
om.init_learning()


Log path = /root/src/aleister/log/logging.log
Log path = /root/src/aleister/log/logging.log


In [ ]:

if arg_dict["execute_mode"] == "prepro":
    om.init_dataGen()
    # period
    train_start = arg_dict["train_start_date"] if "train_start_date" in arg_dict.keys() else None
    train_end = arg_dict["train_end_date"] if "train_end_date" in arg_dict.keys() else None
    valid_start = arg_dict["valid_start_date"] if "valid_start_date" in arg_dict.keys() else None
    valid_end = arg_dict["valid_end_date"] if "valid_end_date" in arg_dict.keys() else None
    test_start = arg_dict["test_start_date"] if "test_start_date" in arg_dict.keys() else None
    test_end = arg_dict["test_end_date"] if "test_end_date" in arg_dict.keys() else None
    om.preprocessing(sym, train_start, train_end, valid_start, valid_end, test_start, test_end)
else:
    if arg_dict["execute_mode"] == "train":
        om.train()
    elif arg_dict["execute_mode"] == "gtrain":
        om.gtrain()
    elif arg_dict["execute_mode"] == "deploy_model":
        om.deploy_best_model()
    elif arg_dict["execute_mode"] == "rpredict":
        om.init_dataGen(remote=True)
        om.realtime_predict()

In [12]:
om.fp

In [25]:
om.init_dataGen(remote=True)
om.realtime_predict()

Log path = /root/src/kuloko/logging.log


NameError: name 'trades' is not defined

In [53]:
realtime_data = om.dg.mq_rpc_client["realtime"].call("")

ChannelWrongStateError: Channel is closed.

In [49]:
import json 

In [61]:
for _key in json.loads(realtime_data).keys():
    print(_key)

orderbook
trade
margin
ticker


In [56]:
pd.DataFrame(json.loads(realtime_data)["orderbook"])

,time,symbol,bids0,bids0_size,bids1,bids1_size,bids2,bids2_size,bids3,bids3_size,...,asks0,asks0_size,asks1,asks1_size,asks2,asks2_size,asks3,asks3_size,asks4,asks4_size
0,20220213112615840000,BTC,4880008.0,0.001,4880006.0,0.02,4880005.0,0.0035,4880001.0,0.0002,...,4880009.0,0.330,4880020.0,0.0019,4880050.0,0.0206,4880070.0,0.0076,4880080.0,0.025
1,20220213112616851000,BTC,4880008.0,0.001,4880006.0,0.02,4880005.0,0.0035,4880001.0,0.0002,...,4880009.0,0.330,4880020.0,0.0019,4880050.0,0.0206,4880070.0,0.0076,4880080.0,0.025
2,20220213112617862000,BTC,4880008.0,0.001,4880006.0,0.02,4880005.0,0.0035,4880001.0,0.0002,...,4880009.0,1.057,4880020.0,0.0019,4880050.0,0.0206,4880070.0,0.0076,4880080.0,0.025
3,20220213112617881000,BTC,4880008.0,0.001,4880006.0,0.02,4880005.0,0.0035,4880001.0,0.0002,...,4880009.0,1.057,4880020.0,0.0019,4880050.0,0.0206,4880070.0,0.0076,4880080.0,0.025
4,20220213112618371000,BTC,4880008.0,0.001,4880006.0,0.02,4880005.0,0.0035,4880001.0,0.0002,...,4880009.0,1.057,4880020.0,0.0019,4880070.0,0.0076,4880080.0,0.0250,4880150.0,2.500
5,20220213112618877000,BTC,4880008.0,0.001,4880006.0,0.02,4880005.0,0.0035,4880001.0,0.0002,...,4880009.0,0.957,4880020.0,0.0019,4880070.0,0.0076,4880080.0,0.0250,4880150.0,2.500
6,20220213112619382000,BTC,4880008.0,0.001,4880006.0,0.02,4880005.0,0.0035,4880001.0,0.0002,...,4880009.0,0.957,4880020.0,0.0019,4880070.0,0.0076,4880080.0,0.0250,4880150.0,2.500
7,20220213112619888000,BTC,4880008.0,0.001,4880006.0,0.02,4880005.0,0.0035,4880001.0,0.0002,...,4880009.0,1.057,4880020.0,0.0019,4880070.0,0.0076,4880080.0,0.0250,4880150.0,2.500
8,20220213112620393000,BTC,4880008.0,0.001,4880006.0,0.02,4880005.0,0.0035,4880001.0,0.0002,...,4880009.0,0.230,4880020.0,0.0019,4880070.0,0.0076,4880080.0,0.0250,4880150.0,2.500


In [58]:
pd.DataFrame(json.loads(realtime_data)["trade"])

""


In [18]:
om.dg.mq_rpc_client["realtime"].mqserver_host

'misaka_rabbitmq'

In [60]:
om.le.mlwriter.client.get_run(best_run[0].info.run_id)

<Run: data=<RunData: metrics={'test_acc': 0.6089201877934273,
 'train_loss': 0.7199654926115008,
 'valid_acc': 0.5555555555555556,
 'valid_loss': 0.727906052271525}, params={'batch_size': '64',
 'loss_fn': 'BCELogitLoss',
 'n_epochs': '15',
 'optimizer': 'adam'}, tags={'mlflow.runName': 'TRAIN_21121520411639568475_0_1_2_3_4_5_6_7_8_9_10_11',
 'mlflow.source.name': 'ipynb',
 'mlflow.user': 'Misaka00001'}>, info=<RunInfo: artifact_uri='/Users/macico/Dropbox/btc/mlruns/2/6f3cf06067a54db289b503f16a9086df/artifacts', end_time=1639568589192, experiment_id='2', lifecycle_stage='active', run_id='6f3cf06067a54db289b503f16a9086df', run_uuid='6f3cf06067a54db289b503f16a9086df', start_time=1639568576581, status='FINISHED', user_id='Misaka00001'>>

In [61]:

om.le.mlwriter.create_model_version("test002_test",run_id=best_run[0].info.run_id)

In [45]:
om.le.mlwriter.client.get_experiment_by_name("test002").experiment_id

'2'

In [46]:
om.le.mlwriter.register_model("test002_test")

/Users/macico/Dropbox/btc/aleister/src/mlflow_writer.py:41: DeprecationWarning:

The 'warn' method is deprecated, use 'warning' instead



In [48]:
mv = om.le.mlwriter.create_model_version("test002_test")

In [50]:
mv.info

AttributeError: 'NoneType' object has no attribute 'info'

In [42]:
import mlflow
from mlflow.entities import ViewType
#serch runs
# query = "params.model = 'CNN' and params.layers = '10' and metrics.`prediction accuracy` >= 0.945"
query = None
run_info = om.le.mlwriter.client.search_runs(experiment_ids=["1"], filter_string=query, run_view_type=ViewType.ACTIVE_ONLY)[0]
uri =  os.path.join(run_info.info.artifact_uri, "model" ) 
mlflow.pytorch.load_model(uri)

SimpleLSTM(
  (lstm): LSTM(7, 32, num_layers=2, batch_first=True)
  (fc): Linear(in_features=32, out_features=3, bias=True)
)

In [43]:
model_info = le.mlwriter.client.get_latest_versions("test_model")[0]
mlflow.pytorch.load_model(model_info.source)

NameError: name 'le' is not defined

In [ ]:
for  i in le.ys:
    print(i)

In [ ]:
import joblib
load_path="/Users/macico/Dropbox/btc_data/model/test001/X_test.jbl"
with open(load_path, mode="rb") as f:
    data = joblib.load(f)
data